# Exercise 1.1
Application for detecting and tracking moving cars from a camera recording using OpenCV and Python.

We are using an algorithm based on frame differencing and background subtraction techniques.

In [7]:
import cv2

source_file = './media/Traffic_Laramie_1.mp4'


In [8]:

def analyze_frame(
        videoSourceStr: str,
        setup_thresholds: bool = False,
    ):
    videoSource = cv2.VideoCapture(videoSourceStr)

    initial_frame = None
    # Threshold to determine if a pixel is different from the initial frame
    threshold = 10
    # Minimum area of a contour to be considered
    contour_area = 2500
    # Start playing video
    while True:
        ret, frame = videoSource.read()
        if not ret:
            if setup_thresholds:
                # Set the video on loop, this will allow us to continue setting the threshold
                videoSource.set(cv2.CAP_PROP_POS_FRAMES, 0)
                continue
            else:
                break
        # As in the instructions the point of interest is roughly the last horizontal half of the video.
        # We will create a ROI (Region of Interest) to focus on that area.
        height, width, _ = frame.shape
        roi = frame[int(height/2):height, 0:width]

        # Background subtraction
        fgmask = cv2.createBackgroundSubtractorMOG2().apply(roi)
        fgmask = cv2.erode(fgmask, None, iterations=2)
        fgmask = cv2.dilate(fgmask, None, iterations=2)
        
        # Gray conversion and noise reduction
        gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (25,25), 0)

        # Save the first frame to compare with the next ones
        if initial_frame is None:
            initial_frame = blur
            continue
        
        # Get the absolute difference between the first frame and the current one
        delta_frame = cv2.absdiff(initial_frame, blur)
        threshold_frame = cv2.threshold(delta_frame, threshold, 255, cv2.THRESH_BINARY)[1]

        # Combine the background subtraction and the threshold frame
        combined_mask = cv2.bitwise_and(fgmask, threshold_frame)

        # Get contours of the threshold frame
        contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw rectangles around the contours
        for contour in contours:
            if cv2.contourArea(contour) < contour_area:
                continue
            x, y, w, h = cv2.boundingRect(contour)
            y = y + int(height/2)
            cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 1)
        
        if setup_thresholds:
            cv2.putText(frame, f'Threshold: {threshold}', (10,20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
            cv2.putText(frame, f'Contour Area: {contour_area}', (10,40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

        cv2.imshow('frame', frame)

        key = cv2.waitKey(1)

        ## Set up control keys
        if key == ord('q'):
            break

        if setup_thresholds:
            # If setting up trhesholds, w and s keys will increase and decrease the threshold value. a and d keys will do the same for the contour area
            if key == ord('w'):
                threshold += 10
            if key == ord('s'):
                threshold -= 10
            if key == ord('a'):
                contour_area -= 100
            if key == ord('d'):
                contour_area += 100

    
    videoSource.release()
    cv2.destroyAllWindows()

def apply_background_subtraction(roi):
    """
    Apply background subtraction to the ROI and return the mask
    """
    fgmask = cv2.createBackgroundSubtractorMOG2().apply(roi)
    fgmask = cv2.erode(fgmask, None, iterations=2)
    fgmask = cv2.dilate(fgmask, None, iterations=2)
    return fgmask

def apply_frame_difference(initial_frame, frame, threshold):
    """
    Apply frame difference to the initial frame and the current frame
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (25,25), 0)
    delta_frame = cv2.absdiff(initial_frame, blur)
    threshold_frame = cv2.threshold(delta_frame, threshold, 255, cv2.THRESH_BINARY)[1]
    return threshold_frame

def draw_contours(contours, frame, contour_area):
    """
    Draw rectangles around the contours
    """
    for contour in contours:
        if cv2.contourArea(contour) < contour_area:
            continue
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(frame, (x,y), (x+w, y+h), (0,255,0), 1)
    return frame

analyze_frame(source_file, True)